In [5]:
from  psaw import PushshiftAPI
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pandas_datareader as web
from dateutil import relativedelta

api = PushshiftAPI()

NASDAQ_df = pd.read_csv('NASDAQ.csv')
AMEX_df = pd.read_csv('AMEX.csv')
NYSE_df = pd.read_csv('NYSE.csv')

frames = [NASDAQ_df, AMEX_df, NYSE_df]
temp = pd.concat(frames)
cols = list(temp.columns)
temp.drop(cols[2:], axis = 1, inplace = True)
tickers = temp['Symbol'].tolist()

In [6]:
def stonk_mentions(start_time, end_time, tickers, m = 1):
    
    start_time_int = int(start_time.timestamp())
    end_time_int = int(end_time.timestamp())
    next_month_time = end_time + relativedelta.relativedelta(months=m)
    
    submissions = api.search_submissions(after = start_time_int, before = end_time_int,
                                subreddit = 'wallstreetbets',
                                filter = ['url', 'author', 'title', 'subreddit'])
    
    df = pd.DataFrame({"Ticker" : tickers})
    
    mentions = []
    for submission in submissions:
        words = submission.title.split()
        cashtags = list(set(filter(lambda word: word.upper().startswith('$'), words)))
        
        if len(cashtags) > 0:
            for cashtag in cashtags:
                tick = cashtag.replace('$','')
                if tick in tickers:
                    mentions.append(tick)
    
    occurances = []
    for tick in tickers:
        try: 
            occurances.append(mentions.count(tick.upper()))
        except:
            pass 
        
    df['Mentions'] = occurances 
    df2 = df[df['Mentions'] > 20]
    df2 = df2.sort_values('Mentions', axis = 0, ascending = False)
    
    df['Start_Time'] = start_time
    df['End_Time'] = end_time
    df2['Start_Time'] = start_time
    df2['End_Time'] = end_time
    
    return df, df2

In [7]:
t = datetime.datetime(2020,11,30)
dates = [t]
for i in range(1,53):
    dates.append(t+relativedelta.relativedelta(weeks=i)) #12 months of time history

In [ ]:
frames_df = []
frames_df2 = []
for i in range(len(dates)):
    df, df2 = stonk_mentions(dates[i], dates[i+1], tickers, 1)
    frames_df.append(df)
    frames_df2.append(df2)
    print(i)

0
1
2
3
4
5
6
7


/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
/opt/anaconda3/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 522
  warnings.warn("Got non 200 code %s" % response.status_code)


8


In [ ]:
result_df = pd.concat(frames_df)
result_df2 = pd.concat(frames_df2)

In [18]:
result_df.to_csv('TickerPullsFull.csv')

In [22]:
frames_df2.reset_index(inplace = True)

367